In [151]:
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn 
patch_sklearn()
from sklearn.svm import SVC
from itertools import combinations
from imports import *
from sklearn.metrics import accuracy_score, f1_score


data = pd.read_csv("C:/Users/kfroo/OneDrive/Desktop/Thesis/Breast Cancer Dataset/dataR2.csv")
target_variable = 'Classification'

data['Classification'] = data['Classification'].map({1:0, 2:1})

def preprocess(data):
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1:]#.squeeze()
    feature_names = X.columns
    scaler = StandardScaler()
    X = scaler.fit_transform(data.drop(columns = [target_variable], axis = 1))
    X = pd.DataFrame(X, columns = feature_names)
    data = pd.concat([X,y] ,axis = 1)
    x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=340, test_size=0.3)
    return x_train, x_test, y_train, y_test, feature_names

x_train, x_test, y_train, y_test, feature_names = preprocess(data)
train_data = pd.concat([x_train, y_train], axis = 1)
test_data = pd.concat([x_test, y_test], axis = 1)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [152]:
def calculate_gini(y):
    """Calculate the Gini index for a given array of labels."""
    p = y.value_counts() / len(y)
    gini = 1 - np.sum(p ** 2)
    return gini
    
def gini_index(decision_function, y):
    """Calculate the Gini index for a given decision function and labels."""
    indices_less_zero = np.where(decision_function < 0)[0]
    indices_greater_zero = np.where(decision_function > 0)[0]

    g1 = y.iloc[indices_less_zero]
    g2 = y.iloc[indices_greater_zero]

    gini1 = calculate_gini(g1)
    gini2 = calculate_gini(g2)

    n = len(y)
    gini_score = (len(g1) / n) * gini1 + (len(g2) / n) * gini2

    return gini_score

In [153]:
def test_split(decision_function, data):
    """Split the data into left and right data frames based on the decision function."""
    indices_less_zero = np.where(decision_function < 0)[0]
    indices_greater_zero = np.where(decision_function > 0)[0]

    left_data = data.iloc[indices_less_zero]
    right_data = data.iloc[indices_greater_zero]
    
    return left_data, right_data
    #return {'left':left_data, 'right':right_data}

In [168]:
#Change this to concider 3 features 

def get_split(data, c):
    c.self = c
    """Find the best Gini split for the given data."""
    best_gini = float('inf')
    best_split = None

    features = data.iloc[:, :-1]
    target = data.iloc[:, -1]

    #Generate all possible combinations of feature pairs
    feature_pairs = list(combinations(features.columns, 3))
    #decision_functions_array = []
    svm_accs = []

    for triple in feature_pairs:
    #    # Select the pair of features
        feature1, feature2, feature3 = triple
        selected_features = features[[feature1, feature2, feature3]]
        #print(selected_features)

        # Train the SVM
        svm = SVC(kernel='linear', C = c)
        svm.fit(selected_features, target)

        #Get accuracy of the svm on test_set
        #y_pred_svm = svm.predict(x_test)
        #svm_accs.append(f"Accuracy: {metrics.accuracy_score(y_test, y_pred_svm)}")

        # Use the decision function to split the data
        decision_function = svm.decision_function(selected_features)
    
        gini = gini_index(decision_function, target)
        #all_gini.append(gini)
 
        if gini < best_gini:
            best_gini = gini
            best_df = decision_function
            best_svm = svm
            best_split = {'feat1': selected_features.columns[0], 
                          'feat2': selected_features.columns[1],
                          'feat3': selected_features.columns[2],
                          'x1': np.around((best_svm.coef_[0][0]), 5), 
                          'x2': np.around((best_svm.coef_[0][1]), 5),
                          'x3': np.around((best_svm.coef_[0][2]), 5),
                          'intercept': np.around(best_svm.intercept_[0], 5)}
            rule = f"{best_split['x1']}*{best_split['feat1']} + {best_split['x2']}*{best_split['feat2']} + {best_split['x3']}*{best_split['feat3']} + {best_split['intercept']} < 0"
            #f"{np.around((best_svm.coef_[0][0]), 5)}*{selected_features.columns[0]} + {np.around((best_svm.coef_[0][1]), 5)}*{selected_features.columns[1]} + {np.around(best_svm.intercept_[0], 5)} < 0 "
        
        groups = test_split(best_df, data)

    return {'best_split':best_split, 'split_rule': rule, 'best_df':best_df, 'best_svm':best_svm, 'groups':groups, 'svm_accs': svm_accs}

In [169]:
# Create a terminal node value
def to_terminal(subset):
    outcomes = list((subset.iloc[:,-1]))
    return max(set(outcomes), key=outcomes.count)

In [167]:
def split(node, max_depth, min_samples, depth, c):
    c.self = c 
    left, right = node['groups']
    del(node['groups'])

    #Check for no-split
    if left.empty or right.empty:
        node['left']  = node['right'] = to_terminal(pd.concat([left, right]))
        return
    
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_samples:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, c)
        split(node['left'], max_depth, min_samples, depth+1)

     # process right child
    if len(right) <= min_samples:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, c)
        split(node['right'], max_depth, min_samples, depth+1)


In [170]:
# Build a decision tree
def build_tree(train, max_depth, min_size, c):
   root = get_split(train)
   split(root, max_depth, min_size, 0, c.self)
   return root

In [159]:
def predict(instance, node):
    if 'split_rule' not in node:
        return node['left']  # Terminal node value
    split_rule = node['split_rule']
    print(split_rule)

In [160]:
# Print a decision tree
def print_tree(node, depth=0):
 if isinstance(node, dict):
    print(depth*'  ', f'Feature:{node["split_rule"]}')
    #print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
    print_tree(node['left'], depth+1)
    print_tree(node['right'], depth+1)
 else:
   print('%s[%s]' % ((depth*'  ', node)))

In [161]:
# Make a prediction with a decision tree
def predict(node, row):
    feat1 = node['best_split']['feat1']
    feat2 = node['best_split']['feat2']
    x1 = node['best_split']['x1']
    x2 = node['best_split']['x2']
    intercept = node['best_split']['intercept']

    score = x1 * row[feat1] + x2 * row[feat2] + intercept 
    if score < 0:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [162]:
grid = {'depth': [i for i in range(1, 11)], 
        'samples': [i for i in range(5, 55, 10)]}

In [163]:
def sd_accuracy(tree, test_data):
    predictions_saved = []
    for row in range(len(test_data)):
        #print(test_data.iloc[row])
        p = predict(tree, test_data.iloc[row])
        predictions_saved.append(p)
    y_pred = list(predictions_saved)
    return round(accuracy_score(y_true, y_pred), 6)

In [171]:
all_trees_in_grid = []
y_true = list(test_data.Classification)
for i in grid['depth']:
    for j in grid['samples']:
        try:
            tree = build_tree(train_data, i, j, c = 0.1)
            print(f"depth: {i} and samples: {j} with accuracy: {sd_accuracy(tree, test_data)}")
            all_trees_in_grid.append(tree)
        except ValueError:
            print(f"depth: {i} and samples: {j}, doesn't work")

TypeError: get_split() missing 1 required positional argument: 'c'

In [ ]:
tree = build_tree(train_data, 4, 5)
#print_tree(tree)

 Feature:1.62404*Glucose + 1.47196*Resistin + -0.40871*MCP.1 + 0.92703 < 0
   Feature:-0.43586*BMI + 0.78886*Insulin + 1.75782*Resistin + -0.29836 < 0
     Feature:-0.0002*Age + 2e-05*BMI + 0.00168*Glucose + -0.99932 < 0
      [0]
      [0]
    [1]
   Feature:-0.00011*Age + -6e-05*BMI + 0.00026*Glucose + 1.00002 < 0
    [1]
    [1]


In [ ]:
y_true = list(test_data.Classification)

In [ ]:
def sd_accuracy(tree, test_data):
    predictions_saved = []
    for row in range(len(test_data)):
        #print(test_data.iloc[row])
        p = predict(tree, test_data.iloc[row])
        predictions_saved.append(p)
    y_pred = list(predictions_saved)
    return round(accuracy_score(y_true, y_pred), 6)

In [ ]:

print(f"accuracy: {round(accuracy_score(y_true, y_pred), 6)}, f1_score: {round(f1_score(y_true, y_pred), 6)}")

#accuracy: 0.714286, f1_score: 0.772727

accuracy: 0.714286, f1_score: 0.772727
